In [ ]:
!pip install spacy nltk pandas tqdm inflect
!python -m spacy download en_core_web_sm
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")

In [ ]:
import os
import json
import random
import re
from pathlib import Path
import pandas as pd
import spacy
from nltk.corpus import wordnet as wn
import nltk
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")
import inflect

In [ ]:
# English inflection helper (used to detect singular/plural of input terms)
p = inflect.engine()

In [ ]:
FRENCH_COMPLEX_PATTERNS = {
    'premises': {
        'all': [
            {'tpl': 'Tous les {a_pl} sont {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Sans exception, tous les {a_pl} sont {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Chaque {a_sg} est {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Tout ce qui est {a_sg_art} est {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Chacun des {a_pl} appartient à {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Si quelque chose est {a_sg_art}, alors c’est {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'La classe des {a_pl} est incluse dans celle des {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'N’importe quel {a_sg} se définit comme {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Il n’existe aucun {a_sg} qui ne soit pas {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Être {a_sg_art} implique d’être {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'La totalité des {a_pl} sont des {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'C’est un fait que chaque {a_sg} est {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Tout membre de {a_pl} est également membre de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Quiconque est {a_sg_art} est forcément {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Les {a_pl} sont universellement classés comme {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'some': [
            {'tpl': 'Quelques {a_pl} sont {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Il existe au moins un {a_sg} qui est {b_sg}.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'On trouve des {a_pl} qui sont {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Au moins quelques {a_pl} sont {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Quelques-uns des {a_pl} sont {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Certains {a_pl} s’avèrent être {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Il arrive que {a_sg_art} soit {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Il est possible de rencontrer des {a_pl} qui sont {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Une partie des {a_pl} est composée de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'On peut identifier des {a_pl} parmi les {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Pas tous, mais au moins certains {a_pl} sont {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Il existe des cas où {a_sg_art} est {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'On note la présence de {b_pl} parmi les {a_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Plusieurs {a_pl} possèdent les caractéristiques de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Il existe des spécimens de {a_pl} qui sont également {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'no': [
            {'tpl': 'Aucun {a_pl} n’est {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Il n’existe aucun {a_sg} qui soit {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Rien de ce qui est {a_sg_art} n’est {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'On ne trouve aucun {a_pl} qui soit {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Jamais les {a_pl} ne sont {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Aucun membre de {a_pl} n’appartient à {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Il est faux qu’un {a_sg} puisse être {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'La classe des {a_pl} est disjointe de celle des {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Absolument aucun {a_sg} ne rentre dans {b_pl}.', 'a_type': 'singular', 'b_type': 'plural'},
            {'tpl': 'En aucun cas un {a_sg} n’est {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Il est exclu que les {a_pl} soient {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Si c’est {a_sg_art}, cela ne peut en aucun cas être {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'On ne saurait trouver un {a_sg} qui soit {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'L’intersection entre {a_pl} et {b_pl} est vide.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Nul {a_sg} n’est admis comme {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'}
        ],
        'some_not': [
            {'tpl': 'Quelques {a_pl} ne sont pas {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Pas tout {a_sg} est {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Certains {a_pl} n’appartiennent pas à {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Tous ne sont pas {b_pl} parmi les {a_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Il y a des {a_pl} qui ne sont pas {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Certains {a_pl} sont dépourvus de la qualité de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Il existe au moins un {a_sg} qui n’est pas {b_sg}.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'On ne peut pas dire que tout membre de {a_pl} soit {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Au moins un cas de {a_sg_art} n’est pas {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Il est faux que la totalité des {a_pl} soient {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Une partie des {a_pl} reste en dehors de la catégorie {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'On peut trouver des {a_pl} qui ne s’avèrent pas être {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Tout ce qui est {a_sg_art} n’est pas forcément {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Des éléments de {a_pl} ne sont pas membres de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Il existe des spécimens de {a_pl} qui ne sont pas {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ]
    },
    'conclusions': [
        'Par conséquent, {conclusion}.',
        'Donc, {conclusion}.',
        'Ainsi, {conclusion}.',
        'On peut en conclure que {conclusion}.',
        'Il s’ensuit que {conclusion}.',
        'Dès lors, {conclusion}.',
        'Il en résulte que {conclusion}.',
        'C’est pourquoi {conclusion}.',
        'Par suite, {conclusion}.',
        'En conclusion, {conclusion}.'
    ]
}

In [ ]:
FRENCH_COMPLEX_PATTERNS_WITH_FOURTH_VAR = {
    'premises': {
        'all': [
            {'tpl': 'Chaque {d_sg} qui est {a_sg_art} est aussi {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Si {d_sg_art} est {a_sg_art}, alors il doit être {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Tous les {d_pl} qui sont {a_pl} sont également {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Sans exception, les {d_pl} en {a_pl} sont {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Chacun des {d_pl} appartenant à {a_pl} est {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'N’importe quel {d_sg} qualifié comme {a_sg_art} est {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Tout objet qui est à la fois {d_sg_art} et {a_sg_art} est aussi {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'La totalité des {d_pl} étant des {a_pl} sont par là même {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Au sein de {d_pl}, tous les {a_pl} appartiennent à {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Pour tout {d_sg}, s’il est {a_sg_art} alors il est nécessairement {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Tout ce qui est simultanément {d_sg_art} et {a_sg_art} est {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Chaque membre de {d_pl} qui fait partie de {a_pl} est {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Quiconque parmi les {d_pl} est {a_sg_art} est forcément {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'plural'},
            {'tpl': 'Il n’y a aucun {d_sg} qui, tout en étant {a_sg_art}, ne soit pas {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Tout exemplaire de {d_sg} possédant la propriété {a_sg} a également {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'}
        ],
        'some': [
            {'tpl': 'Il existe au moins un {d_sg} qui est à la fois {a_sg_art} et {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Quelques {d_pl} sont {a_pl} et {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'On trouve des {d_pl} qui sont {a_pl} et {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Au moins un des {d_pl} est {a_pl} et {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Un {d_sg} existe qui est {a_sg} et {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Certains {d_pl} s’avèrent être simultanément {a_pl} et {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Il est possible d’identifier un {d_sg} qui soit {a_sg_art} et {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Il arrive que certains {d_sg} soient à la fois {a_sg_art} et {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Une portion de {d_pl} présente les caractéristiques de {a_pl} et {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Parmi les {d_pl}, on trouve au moins un exemple qui est à la fois {a_sg} et {b_sg}.', 'a_type': 'plural', 'b_type': 'singular', 'd_type': 'plural'},
            {'tpl': 'On peut repérer des {d_pl} qui satisfont à la fois {a_pl} et {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Quelque {d_sg} est en même temps {a_sg_art} et {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Au sein de la catégorie {d_pl}, il existe des {a_pl} qui sont {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Plusieurs {d_pl} partagent les propriétés de {a_pl} et {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Des éléments de {d_pl} sont identifiés comme {a_pl} et {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ],
        'no': [
            {'tpl': 'Aucun des {d_pl} qui sont {a_pl} n’est {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'On ne trouve aucun {d_pl} qui soit {a_pl} et {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Jamais les {d_pl} en {a_pl} ne sont {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Il n’existe aucun {d_sg} qui soit {a_sg} et {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Aucun {d_sg} en {a_sg} n’est {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Il est impossible qu’un {d_sg} soit à la fois {a_sg_art} et {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Aucun {d_sg} ayant la qualité {a_sg} ne possède également {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'On ne trouve nul {d_sg} qui soit en même temps {a_sg_art} et {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Au sein de {d_pl}, aucun membre de {a_pl} n’est {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Il est exclu que les {d_pl} qui sont {a_pl} soient également {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Il est faux que certains {d_sg} puissent être simultanément {a_sg_art} et {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Nul membre de {d_pl} n’appartient à {a_pl} et {b_pl} à la fois.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Si c’est {d_sg_art} et {a_sg_art}, alors ce ne peut absolument pas être {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Il n’y a aucun moyen qu’un {d_sg} soit à la fois {a_sg} et {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'La catégorie {d_pl} n’admet pas la coexistence de {a_pl} et {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ],
        'some_not': [
            {'tpl': 'Pas tout {d_sg} qui est {a_sg_art} est {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Il y a des {d_pl} qui sont {a_pl} mais pas {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Quelques {d_pl} en {a_pl} ne sont pas {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Au moins certains {d_pl} sont {a_pl} et non {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Il existe un {d_sg} qui est {a_sg} mais pas {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Certains {d_pl} appartenant à {a_pl} n’ont pas la propriété {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Il existe au moins un {d_sg} qui, bien qu’étant {a_sg_art}, n’est pas {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Il n’est pas vrai que chaque {d_sg} étant {a_sg_art} doive aussi être {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'On peut identifier des {d_pl} qui sont {a_pl} mais restent en dehors de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Au sein de la classe {d_pl}, certains {a_pl} ne sont pas {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Tout ce qui est à la fois {d_sg_art} et {a_sg_art} ne s’avère pas être {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Quelque {d_sg} qui est {a_sg} n’est pas {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Il est possible qu’un {d_sg} soit {a_sg} et pourtant ne soit pas {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Plusieurs membres de {d_pl} qui sont {a_pl} ne sont pas membres de {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Une portion de {d_pl} est {a_pl} mais n’est pas {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ]
    }
}

In [ ]:
def get_singular_and_plural(term):
    """Return (singular, plural) using English inflect heuristics (fallback to term).
    """
    term = term.strip()
    try:
        sg_attempt = p.singular_noun(term)
    except Exception:
        sg_attempt = False
    if sg_attempt is not False and sg_attempt is not None:
        sg = sg_attempt
        pl = term
    else:
        sg = term
        try:
            pl = p.plural_noun(term) or term
        except Exception:
            pl = term
    return sg, pl

In [ ]:
def get_article_french(term):
    """Simple French indefinite article heuristic: 'une' if word ends with 'e', else 'un'.
    This is heuristic-only and won't fully handle gender exceptions.
    """
    term = term.strip()
    if term.endswith('e'):
        return 'une ' + term
    return 'un ' + term

In [ ]:
def parse_sentence_for_type_and_terms(sentence):
    sentence = sentence.strip()
    match = re.match(r'All (.*) that are (.*) are (.*).', sentence)
    if match: return 'all', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are not (.*).', sentence)
    if match: return 'some_not', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are (.*).', sentence)
    if match: return 'some', match.group(1), match.group(2), match.group(3)
    match = re.match(r'No (.*) that are (.*) are (.*).', sentence)
    if match: return 'no', match.group(1), match.group(2), match.group(3)
    match = re.match(r'All (.*) are (.*).', sentence)
    if match: return 'all', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are not (.*).', sentence)
    if match: return 'some_not', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are (.*).', sentence)
    if match: return 'some', None, match.group(1), match.group(2)
    match = re.match(r'No (.*) are (.*).', sentence)
    if match: return 'no', None, match.group(1), match.group(2)
    raise ValueError(f'Could not parse basic syllogism type from sentence: {sentence}')

In [ ]:
def create_sub_dict(term_original_subject, term_original_predicate, term_original_fourth, synonym_dict=None):
    sub_map = {}
    sg_a, pl_a = get_singular_and_plural(term_original_subject)
    sub_map['a_sg'] = sg_a
    sub_map['a_pl'] = pl_a
    sub_map['a_sg_art'] = get_article_french(sg_a)
    sg_b, pl_b = get_singular_and_plural(term_original_predicate)
    sub_map['b_sg'] = sg_b
    sub_map['b_pl'] = pl_b
    sub_map['b_sg_art'] = get_article_french(sg_b)
    if term_original_fourth:
        sg_d, pl_d = get_singular_and_plural(term_original_fourth)
        sub_map['d_sg'] = sg_d
        sub_map['d_pl'] = pl_d
        sub_map['d_sg_art'] = get_article_french(sg_d)
    return sub_map

In [ ]:
def safe_lowercase_first(s, sub_map):
    for val in sub_map.values():
        if isinstance(val, str) and s.startswith(val):
            return s
    return s[:1].lower() + s[1:]

In [ ]:
def convert_syllogism_to_complex(syllogism_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    simple_syllogism = syllogism_data['syllogism']
    premises_simple, conclusion_simple = split_syllogism_into_sentences(simple_syllogism)
    complex_premises = []
    for premise in premises_simple:
        p_type, p_fourth, p_subject, p_predicate = parse_sentence_for_type_and_terms(premise)
        sub_map = create_sub_dict(p_subject, p_predicate, p_fourth)
        if p_fourth:
            p_template = random.choice(complex_patterns_data_with_fourth_var['premises'][p_type])
        else:
            p_template = random.choice(complex_patterns_data['premises'][p_type])
        p_complex = p_template['tpl'].format(**sub_map)
        p_complex = p_complex[0].upper() + p_complex[1:]
        complex_premises.append(p_complex)
    c_type, c_fourth, c_subject, c_predicate = parse_sentence_for_type_and_terms(conclusion_simple)
    c_sub_map = create_sub_dict(c_subject, c_predicate, c_fourth)
    if c_fourth:
        c_premise_template = random.choice(complex_patterns_data_with_fourth_var['premises'][c_type])
    else:
        c_premise_template = random.choice(complex_patterns_data['premises'][c_type])
    c_premise_complex = c_premise_template['tpl'].format(**c_sub_map)
    c_premise_complex = safe_lowercase_first(c_premise_complex, c_sub_map)
    c_wrapper_template = random.choice(complex_patterns_data['conclusions'])
    c_complex = c_wrapper_template.format(conclusion=c_premise_complex)
    c_complex = c_complex[:-1] if c_complex.endswith('.') else c_complex
    c_complex = c_complex[0].upper() + c_complex[1:]
    final_complex_syllogism = ' '.join(complex_premises) + ' ' + c_complex
    new_data = syllogism_data.copy()
    new_data['syllogism'] = final_complex_syllogism
    return new_data

In [ ]:
def split_syllogism_into_sentences(syllogism):
    sentences = [s.strip() for s in syllogism.split('.') if s.strip()]
    if len(sentences) < 2:
        raise ValueError(f'Syllogism must contain at least one premise and one conclusion: {syllogism}')
    sentences = [s + '.' for s in sentences]
    conclusion = sentences.pop()
    premises = sentences
    return premises, conclusion

In [ ]:
def process_dataset(input_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    complex_dataset = []
    for item in input_data:
        try:
            complex_item = convert_syllogism_to_complex(item, complex_patterns_data, complex_patterns_data_with_fourth_var)
            complex_dataset.append(complex_item)
        except ValueError as e:
            print(f' Skipping item {item.get(id, 'N/A')} due to parse error: {e}')
            continue
    return complex_dataset

In [ ]:
# Example English input data
sample_data = [
    {
        'id': 'fr-1',
        'syllogism': 'All dogs are animals. All animals are living things. Some dogs are living things.',
        'validity': True,
        'plausibility': True
    },
    {
        'id': 'fr-2',
        'syllogism': 'No fish are mammals. Some mammals are whales. Some whales are not fish.',
        'validity': True,
        'plausibility': True
    }
]
# Run conversion using French templates
complex_syllogism_data = process_dataset(sample_data, FRENCH_COMPLEX_PATTERNS, FRENCH_COMPLEX_PATTERNS_WITH_FOURTH_VAR)
print(json.dumps(complex_syllogism_data, indent=4, ensure_ascii=False))